In [5]:
# Install the DuckDB Python package
import duckdb

# Create path to the original dataset
file_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/itineraries_snappy.parquet"

# Connect to DuckDB and load the dataset
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')")

In [6]:
# Get the number of rows in the dataset
row_count = con.execute("SELECT COUNT(*) FROM flights").fetchall()
print(f"Total rows in the dataset: {row_count[0][0]}")

Total rows in the dataset: 82138753


In [7]:
# Get the column names and data types
columns= con.execute("DESCRIBE flights").fetchall()

for column in columns:
    print(f"Column Name: {column[0]}, Data Type: {column[1]}")

Column Name: legId, Data Type: VARCHAR
Column Name: searchDate, Data Type: VARCHAR
Column Name: flightDate, Data Type: VARCHAR
Column Name: startingAirport, Data Type: VARCHAR
Column Name: destinationAirport, Data Type: VARCHAR
Column Name: fareBasisCode, Data Type: VARCHAR
Column Name: travelDuration, Data Type: VARCHAR
Column Name: elapsedDays, Data Type: BIGINT
Column Name: isBasicEconomy, Data Type: BOOLEAN
Column Name: isRefundable, Data Type: BOOLEAN
Column Name: isNonStop, Data Type: BOOLEAN
Column Name: baseFare, Data Type: DOUBLE
Column Name: totalFare, Data Type: DOUBLE
Column Name: seatsRemaining, Data Type: BIGINT
Column Name: totalTravelDistance, Data Type: DOUBLE
Column Name: segmentsDepartureTimeEpochSeconds, Data Type: VARCHAR
Column Name: segmentsDepartureTimeRaw, Data Type: VARCHAR
Column Name: segmentsArrivalTimeEpochSeconds, Data Type: VARCHAR
Column Name: segmentsArrivalTimeRaw, Data Type: VARCHAR
Column Name: segmentsArrivalAirportCode, Data Type: VARCHAR
Column N

In [8]:
# Filter dataset for LAX flights in June, July, and August (Summer)
con = duckdb.connect()
con.execute("CREATE TABLE flights AS SELECT * FROM read_parquet('" + file_path + "')") 

con.execute("""
    CREATE TABLE summer_selected_airports AS 
    SELECT 
        legId,
        searchDate,
        flightDate,
        startingAirport,
        destinationAirport,
        fareBasisCode,
        travelDuration,
        isBasicEconomy,
        isRefundable,
        isNonStop,
        baseFare,
        totalFare,
        seatsRemaining,
        totalTravelDistance,
        segmentsDepartureTimeEpochSeconds,
        segmentsArrivalTimeEpochSeconds,
        segmentsArrivalAirportCode,
        segmentsDepartureAirportCode,
        segmentsAirlineName,
        segmentsDurationInSeconds,
        segmentsDistance,
        segmentsCabinCode
    FROM flights
    WHERE EXTRACT(MONTH FROM CAST(flightDate as DATE)) IN (6, 7, 8)
    AND (startingAirport IN ('LAX'))
""")

# Get the count of rows in the filtered dataset
row_count = con.execute("SELECT COUNT(*) FROM summer_selected_airports").fetchall()
print(f"Total rows in LAX summer flights dataset: {row_count[0][0]}")

# Export the filtered dataset to a Parquet file
output_path = "/Users/camryncurtis/Downloads/Capstone Data Preprocessing/summer_lax_snappy.parquet"
con.execute(f"COPY (SELECT * FROM summer_selected_airports) TO '{output_path}' (FORMAT PARQUET, COMPRESSION SNAPPY)")

# Close the DuckDB connection
con.close()

Total rows in LAX summer flights dataset: 4329731
